

# Personalization Project - Part 1 HW2
**Team:**  Phani Valasa & Harish Visweswaran 

**Data Set:**  Taste Profile DataSet by The Echo Nest as part of the Million Song Dataset.

### Business Abstract

The objective of this Case study is improvise online music provider services. Many of the Music service providers today offer a freemium model with the majority of its users streaming music to their mobiles or desktop via apps or web browsers. Users subscribe for free get ads between tracks which are part of the revenue model. Users of the free service encounter audio ads every five or six songs, or approximately three minutes of advertising for every hour of listening. There are also paid services with uninterrupted streaming.

For this business plan to work and remain relevant, it is imperative that right recommendations on songs are provided to users for customer loyalty and stickiness with the app. The problem is more complex than it seems.  Challenges are with the sparse dataset pertaining to user listening and identification of similarities between users, songs and artists.

The below case study accesses the data sets available from Kaggle, MSD, LasROSA and Last.fm to evaluate the recommendation models and build a recommendation engine for our Music streaming company.

### Objectives and Cosiderations

To provide meaningfully relevant Music recommendations for users based on their listening behaviour.

We are trying to optimize on several parameters:

* Neighborhood size, similarity measure, model size for neighborhood based methods and number of latent factors for model based methods

The metrics of choice are:
* MAE: MSE is important for us as we are trying to predict the actual listening count in the neighborhood based method
* RMSE: For a dataset that has a large range of listen counts (unlike in ratings dataset), prioritizing RMSE might lead to conservative models. This is something we wanted to avoid. However, we still report RMSE as a secondary metric.
* Top K recommendations: In our model based method, we compute the top K performance across number of latent factors


### Data Sets

Our dataset is the taste profile dataset that is curated from a set of applications including the last fm dataset. The core dataset is the listen count of songs by user. User id, song id and listen count. It also has metadata available - the song name and the artist name. 

Other information about the songs is available for lookup via the million song dataset but for part 1 of the project, only the core dataset will be used.

The core data is the Taste Profile Subset released by The Echo Nest as part of the Million Song Dataset. It consists of 48MM triplets (user ID, song ID, play count), 384K songs and over 1MM users.


### Sampling of Data

For this part of the homework, we kept the data set to be under 10,000 users and 10 songs as suggested in the instructions. This helped to gain a better understanding of the algorithms by running multiple iterations. Since there are 1.1MM users and 390K users in the original data, we first chose the top 10,000 users with the most songs. And then created a data set with the top 100 songs within this. This decreases the probability of randomly holding out the rating for an user who only listened to one song. After sampling the data is 11% dense.

The sampling also maintains random seeds in case we need to replicate a run and can also specify new random seeds to get entirely new random data for cross-validations.

### Cross Validation Methods

The data was split by sampling user-song listen counts at random. We held out 20% for the test set and split the remainder of the 80% into training and validation - again using an 80-20 ratio. As the data was sampled at random, it is possible to have some users be part of only the test set. In such cases, we make use of the item-baseline predictions for the user


## Neighborhood Based Approaches

We had a choice between user based and item based collaborative filtering methods. In our dataset, the number of datapoints (listen counts) per song is greater than the number of datapoints per user. Using the item based collaborative method would allow us to have more data which likely leads to better similarity measures between songs. 

We designed a class called ItembasedCF that has a scikit-learn model type workflow. The model is instantiated using the item neighborhood size and the similarity measure. The model is then trained using the fit method. Once trained, the model can access the predict_individual method to predict the rating for one user and item at a time or alternatively, use the predict method to predict the ratings for the entire test set at once.

Writing the classes and methods from scratch gave us a good understanding of the nuances around item-item CF methods. Moreover, we learned ways to make our code more efficient. 


Similarity Measures:
Four similarity measures were included in the module:
* Cosine
* Cosine Adjusted
* Euclidian
* Pearson

Baseline Measures:
The item based CF model for our songs data works as follows:
For each song, the nearest neighbors are calculated using a decided similarity measure. This leverages the listening count data for these songs across all users. For the user in question, the listening count for a particular song is computed using a weighted average of the listening count of other similar songs. If the user has not listened to any of the songs in the neighborhood set, then the baseline listening count is used as a prediction.

For the baseline, we have four different measures:
* Global Average
* Item Average
* User Average
* Item Adjusted User Average

Here, we default to using the item average as it generally has a higher number of data points. However, it will be interesting to check how the other similarity measures perform.

#### Choice of metric for the neighborhood based method

For the neighborhood based model, we chose Mean Absolute Error as the primary metric. Our goal is to be able to predict the listening count of various songs for different users. This will help us recommend songs that are likely to be listened to many times by the user. In turn, we believe this drives app engagement and user satisfaction. The reason we prioritized MAE over RMSE is that RMSE penalizes larger errors very severely. For a dataset that has a large range of listen counts (unlike in ratings dataset), prioritizing RMSE might lead to conservative models. This is something we wanted to avoid. However, we still report RMSE as a secondary metric.

#### MAE and RMSE vs Neighborhood size and similarity measure

In the below plot of MAE for various similarity measures across different values of neighborhood size - ranging from 1 to 25, it can be seen that MAE decreases with increase in neighborhood size. We also notice that Euclidian Distance performs the worst among the similarity measures tested. Cosine Adjusted Similarity provides the best performance for our dataset. It also has the fastest drop in MAE with increasing K. We see the lowest MAE for a value of _K = 20_. However, larger values of K take longer to run. 

![image1](images/image1.png?raw=true "Image1")

In the RMSE plot, howerever, the trend on RMSE is not very clear. The RMSE generally increases and then decreases with increase in neighborhood size. While the best RMSE is at k=1, a k value close to 20 for the cosine adjusted similarity does better than the baseline and aligns well with the MAE plot. We believe that MAE is more important than RMSE. The RMSE value for implicit feedback with a big range of listen counts is hard to get very accurate unless we are very conservative and we don't want to limit ourselves to that.

![image2](images/image2.png?raw=true "Image2")

#### Running time for different neighborhood sizes
Since larger values of K are expected to take longer, we wanted to check the effect of K on model running time. We notice from the below chart that running time increases linearly with K. For applications that provide real-time predictions, there needs to be a careful consideration of the performance vs time balance.

![image3](images/image3.png?raw=true "Image3")

#### Coverage
We also wanted to understand if the model performs better (lower MAE) for users who have a larger number of data points. What we see is that users who have a larger number of data points in the test set tend to have a higher mean listening count. This is expected - generally, users who listen to more songs also listen more frequently because they are engaged with the app. However, the MAE is bound to be larger when the listening counts are higher and so we adjust the MAE by dividing it by the mean listening count (results in something along the lines of percentage error). We then plot the average adjusted MAE against the number of data points (predictions) in the test set and we can see that as the number of predictions increases, the adjusted MAE value decreases. This indicates to us that the model does better for users who have a larger number of data points. So the coverage of the model has to be balanced with accuracy.

![image5](images/image5.png?raw=true "Image5")

#### Example Prediction

Here is an example of the predictions from the model with K=20 and cosine similarity measure. This is for one user across all items in the test set:

![image4](images/image4.png?raw=true "Image4")

While the actual predictions are not exact, we can see a clear directional relationship. Songs which were listened to more times actually have a higher prediction, on average. Surprisingly, the model does decently well even when the songs listened by the user where across many genres. But this is just one example.


#### Effect of training dataset size on accuracy and model running time

Based on the below graph, we can see that the MAE decreases with a larger model size (item set in this case). This implies that there might be a performancs vs time tradeoff. Also, it's clear that the model MAE is lower than the baseline but they track each other closely for different item set sizes

![image6](images/image6.png?raw=true "Image6")

Again, it can be clearly seen from the below graph that the running time increases linearly with increase in model size (items in this case). For applications that require a fast response, the additional accuracy gained by using a larger model has to be weighed against the significant increase in run time

![image7](images/image7.png?raw=true "Image7")

##### Alternative Design Choices

Implicit data is inherently challenging because it does not have a rating scale like explicit ratings. Hence, interest in a song can be defined in many ways. Instead of using the raw listening counts, we can bucket them into ratings and treat the problem as an explicit data problem (if we don't need to predict the exact listening counts). We also could try out the log transformation on our listening count as they have a long tail.

We can also look at quality of predictions. We can apply a threshold to make sure item similarity is computed only if the items have atleast N users in common and see how that affects the model accuracy.

Please take a look at this [notebook](notebooks/Item-Item CF.ipynb? "ItemItemCF") for some details about the data (distributions, medians, etc). It also has the code for all the above listed graphs and analyses.

In the next section, we will be exploring model based methods. Some of the above concerns are addressed the model based approach. 




## Model Based Approaches

We have implemented two different types of algorithms using Matrix Factorization from Implicit and Non-negative Matrix Factorization (NMF) from Surprise.

Please refer to the detailed analysis on Implicit ALS Matrix Factorization method in this [notebook](notebooks/implicit_als.ipynb? "Implicit"). We covered the aspects of Cross Validation, hyper parameter tuning, performance with increases model sizes and Coverage metrics. Also shown are the affects of increasing factors on the SSE and Coverage metrics. SSE in this case has proven to be of no significant value as the predictions tend to be around 0-1 range. Please refer to the detailed analysis on Implicit ALS Matrix Factorization method in 'implicit_als.ipynb'. 

We have also used an alternate approach using NMF from Surprise package. We have tried several approaches using this package, detailed results of which are in this [notebook](notebooks/surprise_nmf.ipynb? "Surprise")

* Reduced the listen_count using minmax scaler
* Use the min and max of the listen count as a range
* use Grid search to Cross Validate and evaluate the performance of various hyper parameters.
* Perform the analysis on model performance with varying latent factors
* Create the baseline performance using this package and compare the improvements
* Measure the Scaling with increased sampling sizes


## Next Steps

For phase 1 of the project, we haven’t used any of the song metadata information available via the million song dataset. We believe song attributes like genre will contribute to good recommendations. We only explored neighborhood based and matrix factorization methods for the model. There are many other methods like factorization machines that we believe has the potential to produce better results with the additional information about songs. 

As stated above, implicit data is inherently challenging because it does not have a rating scale like explicit ratings. As a consequence, we can't really assume that the user doesn't like all the songs that he hasn't listened to. In explicit ratings data, the items that a user does not like will have lower ratings. Defining what a user likes is somewhat harder with implicit data. We'd like to address this in phase 2 of our project.

Lastly, the songs that the user hasn't listened need not be missing at random. We'd like to account for this fact in our future analyses


## Conclusion

The case study helped us understand the application of recommender systems in the music industry. Based on our exploration, we believe that we have a model that performs better than the baseline and hence can possibly be used to make recommendations(if there is no recommender system being used currently). We also believe that there are a lot of better methods to explore and having additional metadata about songs should help push the needle in terms of accuracy. Also, if the music streaming app had the ability to collect user data, that would enhance the recommender system.

Some learnings:
* Exploring the parameter space is important - for item based approaches, looking at neighborhood size, similarity measures, model size and for matrix factorization methods, looking at number of latent factors is paramount to obtain a good recommmender system.
* Larger neighborhoods work better but the accuracy (MAE in our case) gets worse if neighborhoods are too large
* There is a tradeoff between accuracy and run time and it needs to be seen whether the speed of the model is acceptable
* Users with more data points tend to have better recommendations and hence it is important to assess whether there are enough users with an acceptable amount of data

If we choose to roll this out, we need to use a phased deployment method and also have a testing framework to assess whether the recommender system is doing well.

Finally, we had a great experience working on this project. While writing out the scripts for item-based model ourselves took longer than expected, it was a very enjoyable experience. We learned more about classes, class-methods, static-methods and instance-methods. We also explored standard recommender systems packages like Surprise and Implicit for our analyses. We are looking forward to trying more advanced methods in phase 2.
